In [1]:
CENTRALIZED_TRAINING_DIRPATH = '/home/eltons-pc/Backups/Logs/centralized_sc_depth/'
CENTRALIZED_TRAINING_ID = '02_05_2023' # 100 epochs with 1000 train batches and all val batches

FEDERATED_TRAINING_ID_IID_1 = '24_04_2023_23:17:18' # GOOD run_10_participants_5_per_round_30_rounds_3_local_epochs_1000_local_train_and_all_local_val_batches_sequential_iid_8_workers.log
FEDERATED_TRAINING_ID_IID_2 = '03_05_2023_00:34:11' #  GOOD run_10_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_sequential_iid_8_workers.log
FEDERATED_TRAINING_ID_IID_3 = '07_05_2023_10:37:28' #  GOOD run_5_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_sequential_iid_8_workers.log
FEDERATED_TRAINING_ID_BY_DRIVE_1 = '27_04_2023_22:07:45'  # GOOD run_10_participants_5_per_round_30_rounds_3_local_epochs_1000_local_train_and_all_local_val_batches_random_by_drive_eager_sort_8_workers.log
FEDERATED_TRAINING_ID_BY_DRIVE_2 = '02_05_2023_11:16:39' # GOOD run_10_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_random_by_drive_eager_sort_8_workers.log
FEDERATED_TRAINING_ID_BY_DRIVE_3 = '06_05_2023_17:22:08' # GOOD run_5_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_random_by_drive_eager_sort_8_workers.log

# AFTER FEDAVG FIX

FEDERATED_TRAINING_ID_IID_4 = '09_05_2023_10:30:05' # GOOD run_10_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_sequential_iid_8_workers.log
FEDERATED_TRAINING_ID_IID_5 = '10_05_2023_21:58:21' #  run_9_participants_3_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_sequential_iid_8_workers.log
FEDERATED_TRAINING_ID_BY_DRIVE_4 = '08_05_2023_23:18:52' # GOOD run_10_participants_5_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_random_by_drive_eager_sort_redistribute_remaining_8_workers.log
FEDERATED_TRAINING_ID_BY_DRIVE_5 = '09_05_2023_22:03:14' # GOOD run_9_participants_3_per_round_30_rounds_1_local_epochs_1000_local_train_and_all_local_val_batches_random_by_drive_eager_sort_redistribute_remaining_8_workers.log

FEDERATED_TRAINING_DIRPATH = '/home/eltons-pc/Logs/federated-sc-depth/'
TRAINING_IDS = [
    FEDERATED_TRAINING_ID_IID_1, FEDERATED_TRAINING_ID_IID_2, FEDERATED_TRAINING_ID_IID_3, FEDERATED_TRAINING_ID_IID_4, FEDERATED_TRAINING_ID_IID_5,
    FEDERATED_TRAINING_ID_BY_DRIVE_1, FEDERATED_TRAINING_ID_BY_DRIVE_2, FEDERATED_TRAINING_ID_BY_DRIVE_3, FEDERATED_TRAINING_ID_BY_DRIVE_4, FEDERATED_TRAINING_ID_BY_DRIVE_5,
    CENTRALIZED_TRAINING_ID
]
TRAINING_LABELS = [
    "Federated Training (IID) - 10P5PR3LE", "Federated Training (IID) - 10P5PE1LE", "Federated Training (IID) - 5P5PE1LE", "Federated Training (IID) - 10P5PE1LE - Weighted", "Federated Training (IID) - 9P3PE1LE - Weighted",
    "Federated Training (Non-IID) - 10P5PR3LE", "Federated Training (Non-IID)  - 10P5PE1LE", "Federated Training (Non-IID) - 5P5PE1LE", "Federated Training (Non-IID) - 10P5PE1LE - Weighted", "Federated Training (Non-IID) - 9P3PE1LE - Weighted",
    "Centralized Training"
]
TRAINING_CENTRALIZED = [False, False, False, False, False, False, False, False, False, False, True]
ROUND_CAP = 12
NUM_CLIENTS = 10
MODEL_SIZE_MB = 111.417 # estimated by pytorch

In [2]:
import os
from util import get_dir_size
import pandas as pd
TRAIN_DATASET_DIR = '/home/eltons-pc/Datasets/kitti/training'
FORMATS = ['jpg', 'png']
total_files, total_size_gb, filtered_files, filtered_size_gb = get_dir_size(TRAIN_DATASET_DIR, FORMATS)
df_columns = ['Total Files', 'Total Size (GB)', f'{FORMATS} Files', f'{FORMATS} Size (GB)']
df_rows = [total_files, total_size_gb, filtered_files, filtered_size_gb]
df = pd.DataFrame({'': df_columns, 'Value': df_rows})
df

,,Value
0,Total Files,135080.00
1,Total Size (GB),15.56
2,"['jpg', 'png'] Files",90074.00
3,"['jpg', 'png'] Size (GB)",12.05


In [3]:
from util import get_federated_training_charts, standardize_fig
import plotly.graph_objects as go

test_loss_figures = []
communication_cost_figures = []
test_loss_by_communication_cost_figures = []
for i in range(len(TRAINING_IDS)):
    if TRAINING_CENTRALIZED[i]:
        continue
    test_loss_fig, communication_cost_fig, _, _, test_loss_by_communication_cost_fig = get_federated_training_charts(FEDERATED_TRAINING_DIRPATH, ROUND_CAP, 
                                                                                                                     TRAINING_IDS[i], TRAINING_LABELS[i], 
                                                                                                                     TRAINING_CENTRALIZED[i], filtered_size_gb, 
                                                                                                                     1, MODEL_SIZE_MB)
    test_loss_figures.append(test_loss_fig)
    communication_cost_figures.append(communication_cost_fig)
    test_loss_by_communication_cost_figures.append(test_loss_by_communication_cost_fig)
combined_test_loss_figures = go.Figure()
combined_communication_cost_figures = go.Figure()
combined_test_loss_by_communication_cost_figures = go.Figure()
for fig in test_loss_figures:
    for trace in fig.data:
        combined_test_loss_figures.add_trace(trace)
combined_test_loss_figures.update_layout(title="Lowest Global Validation Loss up to Round", xaxis_title="Round", yaxis_title="Global Validation Loss")
combined_test_loss_figures = standardize_fig(combined_test_loss_figures)
#combined_test_loss_figures.update_layout(legend=dict(x=0.6, y=0.9)) 
combined_test_loss_figures.show()
#combined_test_loss_figures.write_image("loss_by_round.pdf")
for fig in communication_cost_figures:
    for trace in fig.data:
        combined_communication_cost_figures.add_trace(trace)
combined_communication_cost_figures.update_layout(title="Communication Cost for Lowest Validation Loss up to Round", xaxis_title="Round", yaxis_title="Communication Cost (in Gigabytes)")
combined_communication_cost_figures = standardize_fig(combined_communication_cost_figures)
#combined_communication_cost_figures.update_layout(legend=dict(x=0.6, y=0.1)) 
combined_communication_cost_figures.show()
#combined_communication_cost_figures.write_image("communication_cost_by_round.pdf")
for fig in test_loss_by_communication_cost_figures:
    for trace in fig.data:
        combined_test_loss_by_communication_cost_figures.add_trace(trace)
combined_test_loss_by_communication_cost_figures.update_layout(title="Lowest Validation Loss By Communication Cost (in Gigabytes)", xaxis_title="Communication Cost (in Gigabytes)", yaxis_title="Global Validation Loss")
combined_test_loss_by_communication_cost_figures = standardize_fig(combined_test_loss_by_communication_cost_figures)
#combined_test_loss_by_communication_cost_figures.update_layout(legend=dict(x=0.6, y=0.9)) 
combined_test_loss_by_communication_cost_figures.show()
#combined_test_loss_by_communication_cost_figures.write_image("loss_by_communication_cost_by_round.pdf")

AssertionError: federated_training_dirpath does not exist!

In [ ]:
from util import get_federated_training_charts, get_centralized_training_charts, standardize_fig

test_loss_figures = []
communication_cost_figures = []
test_loss_by_communication_cost_figures = []
for i in range(len(TRAINING_IDS)):
    if TRAINING_CENTRALIZED[i]:
        test_loss_by_training_step_fig, communication_cost_by_training_step, test_loss_by_communication_cost_fig = get_centralized_training_charts(
            CENTRALIZED_TRAINING_DIRPATH, TRAINING_IDS[i], TRAINING_LABELS[i], filtered_size_gb, 1, MODEL_SIZE_MB, NUM_CLIENTS
        )
    else:
        _, _, test_loss_by_training_step_fig, communication_cost_by_training_step, test_loss_by_communication_cost_fig = get_federated_training_charts(
            FEDERATED_TRAINING_DIRPATH, ROUND_CAP, TRAINING_IDS[i], TRAINING_LABELS[i], TRAINING_CENTRALIZED[i], filtered_size_gb, 1, MODEL_SIZE_MB
        )
    test_loss_figures.append(test_loss_by_training_step_fig)
    communication_cost_figures.append(communication_cost_by_training_step)
    test_loss_by_communication_cost_figures.append(test_loss_by_communication_cost_fig)
combined_test_loss_figures = go.Figure()
combined_communication_cost_figures = go.Figure()
combined_test_loss_by_communication_cost_figures = go.Figure()
for fig in test_loss_figures:
    for trace in fig.data:
        combined_test_loss_figures.add_trace(trace)
combined_test_loss_figures.update_layout(title="Lowest Global Validation Loss up to Training Step", xaxis_title="Training Step", yaxis_title="Global Validation Loss")
combined_test_loss_figures = standardize_fig(combined_test_loss_figures)
#combined_test_loss_figures.update_layout(legend=dict(x=0.55, y=0.9)) 
combined_test_loss_figures.show()
#combined_test_loss_figures.write_image("loss_by_steps.pdf")
for fig in communication_cost_figures:
    for trace in fig.data:
        combined_communication_cost_figures.add_trace(trace)
combined_communication_cost_figures.update_layout(title="Communication Cost for Lowest Validation Loss up to Training Step", xaxis_title="Training Step", yaxis_title="Communication Cost (in Gigabytes)")
combined_communication_cost_figures = standardize_fig(combined_communication_cost_figures)
#combined_communication_cost_figures.update_layout(legend=dict(x=0.55, y=0.1)) 
combined_communication_cost_figures.show()
#combined_communication_cost_figures.write_image("communication_cost_by_steps.pdf")
for fig in test_loss_by_communication_cost_figures:
    for trace in fig.data:
        combined_test_loss_by_communication_cost_figures.add_trace(trace)
combined_test_loss_by_communication_cost_figures.update_layout(title="Lowest Validation Loss By Communication Cost (in Gigabytes)", xaxis_title="Communication Cost (in Gigabytes)", yaxis_title="Global Validation Loss")
combined_test_loss_by_communication_cost_figures = standardize_fig(combined_test_loss_by_communication_cost_figures)
#combined_test_loss_by_communication_cost_figures.update_layout(legend=dict(x=0.55, y=0.9)) 
combined_test_loss_by_communication_cost_figures.show()
#combined_test_loss_by_communication_cost_figures.write_image("loss_by_communication_cost_by_steps.pdf")